In [1]:
import pandas as pd
import src.scripts as src
from multiprocessing import Pool
!pip install tqdm
from tqdm import tqdm_notebook

    100% |████████████████████████████████| 51kB 12.7MB/s ta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
df_id = src.read_from_efs('df_id.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
df_id.shape

(90624128, 7)

In [4]:
PartD_id_labeled_npi = src.read_from_efs('PartD_id_labeled_npi.csv')

In [5]:
PartD_id_labeled_npi.head()

,npi,last/org name,first name,city,specialty,exclusion_method,exclusion_year
0,1003042441,MANZO,MARK,GRAFTON,Physician Assistant,npi,2016.0
1,1003801044,AFZAL,FIAZ,KENNER,Internal Medicine,npi,2014.0
2,1003809195,MACKEY,JOSEPH,BLUE SPRINGS,Family Practice,npi,2014.0
3,1003811167,SORELL,PAUL,HUNTINGTON,Pain Management,npi,2017.0
4,1003817743,ECKMAN,CHARLES,OKLAHOMA CITY,Interventional Pain Management,npi,2017.0


In [6]:
def build_targets(excluded_npi_year_pair):
    excluded_npi = excluded_npi_year_pair[0]
    excluded_year = excluded_npi_year_pair[1]
    exclusion_df = df_id[(df_id['npi'] == excluded_npi) & (df_id['year'] <= excluded_year)]
    return exclusion_df

In [7]:
excluded_npi_year_pairs = PartD_id_labeled_npi[['npi','exclusion_year']].astype(int).values

In [8]:
print("Number of available CPU cores: ")
!cat /proc/cpuinfo | grep processor | wc -l

Number of available CPU cores: 
16


In [10]:
processors = 16
if __name__ == '__main__':
    tasks = excluded_npi_year_pairs
    function = build_targets
    with Pool(processes=processors)as p:
        targets = list(tqdm_notebook(p.imap(function, tasks), total=len(tasks)))

In [11]:
# unpack
excluded_df = pd.concat(targets, axis=0)
src.save_to_efs(excluded_df, 'excluded_df.csv')

In [ ]:
# extract `year_npi` pairs
excluded_year_npi_pairs = excluded_df[['year','npi']].drop_duplicates().values.tolist()
target_df = df_id[['year','npi']]

In [ ]:
def get_excluded_indices(excluded_year_npi_pair):
    return df_year_npi_pairs[(target_df['year']==excluded_year_npi_pair[0]) & \
                             (target_df['npi']==excluded_year_npi_pair[1])].index.tolist()

In [ ]:
print("Number of available CPU cores: ")
!cat /proc/cpuinfo | grep processor | wc -l